# `1.2 Accelerometer & Gyroscope Configurations 🔄`
## <span style="color:tomato"> Overview </span>
The MPU-6000 is a motion tracking device that combines a 3-axis gyroscope, 3-axis accelerometer, and a digital motion processor all in a small 4x4x0.9mm package. The MPU-6000 features three 16-bit analog-to-digital converters (ADCs) for digitizing the gyroscope outputs
and three 16-bit ADCs for digitizing the accelerometer outputs. An on-chip 1024 Byte First In First Out (FIFO) buffer is there where new samples are added at the end and older samples are overwritten when the buffer is full. This buffer helps lower system power consumption by allowing the system processor to read the sensor data in bursts and then enter a low-power mode as the MPU collects more data.

**<u>Note:</u>**
**GLEE currently has no plans to add functionality to the code such that the gyroscope can be used. We will only be using the accelerometer for now, the inclusion of the gyroscope is informational only**

---

## <span style="color:tomato"> Configurations </span>
#### The sensor has several primary configurable modes: 
- <span style="color:orange"> **Accelerometer Only**</span>: The sensor offers orientation detection and signaling, tap detection for user input and gesture recognition, and user-programmable interrupts based on acceleration events. The operating current is 500 µA.

- <span style="color:orange"> **Accelerometer Low Power**</span>: Wakes the sensor to take a single accelerometer sample  at a given frequency. The current at 1.25Hz is 10µA, at 5Hz is 20µA, at 20Hz is 70µA , and at 40Hz is 140µA. These values indicate the amount of current consumed by the accelerometer when operating in the low power mode at specific output data rates (Hz). Frequency refers to the number of occurrences of a repeating event per unit of time, typically measured in hertz (Hz).

- <span style="color:orange"> **Gyroscope Only**</span>:  The sensor works as a gyroscope only. The gyroscope features integrated 16-bit ADCs for simultaneous sampling, enhanced temperature stability to reduce the need for user calibration, and improved low-frequency noise performance. It also offers a digitally-programmable low-pass filter. The operating current is 3.6mA and has a standby current of 5µA. Additionally, the gyroscope comes factory-calibrated for reliable performance.

- <span style="color:orange">**Accelerometer and Gyroscope**</span>: The sensor works as both an accelerometer as well as a gyroscope. The gyroscope and accelerometer take samples at different rates. The data value assumes a sample rate of the accelerometer, 1 kHz.

- <span style="color:orange">**Gyroscope DMP**</span>: Similar to a Gyroscope configuration with an additional Digital Motion Processor (DMP) engine that supports 3D Motion Processing and gesture recognition algorithms. Due to the DMP, the sensor has built-in capabilities to process motion data and recognize specific gestures or motion patterns.

- <span style="color:orange">**Accelerometer and Gyroscope DMP**</span>:  This is similar to the Accelerometer and Gyroscope configuration with an additional Digital Motion Processor (DMP) engine that supports 3D Motion Processing and gesture recognition algorithms. Due to the DMP, the sensor has built-in capabilities to process motion data and recognize specific gestures or motion patterns.

- <span style="color:orange">**Shutdown**</span>: The sensor enters a shutdown mode where it stops the ongoing process.


When everything is enabled the maximum operating current is 3.9 mA.

-  Power assumes a voltage of 3.3V, and data assumes there are no bottlenecks that would prevent the LunaSat from recording every measurement such as the LunaSat's loop rate.

---

## <span style="color:tomato"> Creating Configurations </span>

We'll generate a model of the sensor's power and data usage with these configurations, and ultimately generate a power and data usage model for the entire LunaSat. These configurations will be passed into the flight code of the LunaSats in order to appropriately setup sensors for the mission.

From the following options, create several configurations by choosing the mode, the low power frequency (in Hz), and the digital low pass (a 3-bit binary number). 

- <span style="color:orange">**Modes**</span>: {"ACCELEROMETER", "ACCELEROMETER_LOW_POWER", "GYROSCOPE", "GYROSCOPE_DMP", "ACCELEROMETER_AND_GYROSCOPE", "ACCELEROMETER_AND_GYROSCOPE_DMP", "SHUTDOWN"}
- <span style="color:orange">**Low Power Frequency (in Hz)**</span>: {0, 1.25, 5, 20, 40} 
    - Low Power Frequency are used for the ACCELEROMETER_LOW_POWER mode only
    - Note that for modes other than ACCELEROMETER_LOW_POWER the Low Power Frequency parameter will be 0.  
- <span style="color:orange">**Digital Low Pass**</span>: Configures the digital low pass filter settings on the sensor. Setting digital_low_pass to anything other than "000" or "111" will lower sample rate of the sensor by setting the Gyroscope Output Rate to 1kHz. 
    - {"000", "001", "010", "011", "100", "101", "110", "111"}
- <span style="color:orange">**Sample rate divisor**</span>: {n}
  - Is a parameter to lower the sampling rate of the sensor. This value ranges from 0 to 255.

Certain configurations do not utilize all available settings. Below are the configuration formats for the different possible modes you can choose from. Anything with {} braces indicates a parameter you can change.

- <span style="color:orange">("SHUTDOWN", 0, "000", 0)</span>
- <span style="color:orange">("ACCELEROMETER_LOW_POWER", {Low Power Frequency}, {Digital Low Pass (any 3 bit number other than "000", "111")}, {Sample rate divisor})</span>
    * Example - ("ACCELEROMETER_LOW_POWER", 5, "011", 3)
- <span style="color:orange">("ACCELEROMETER", 0, {Digital Low Pass (any 3 bit number other than "000", "111")}, {Sample rate divisor})</span>
    * Example - ("ACCELEROMETER", 0, "100" , 40)
- <span style="color:orange">("ACCELEROMETER_AND_GYROSCOPE", 0, {Digital Low Pass (any 3 bit number other than "000", "111")}, {Sample rate divisor})</span>
    * Example - ("ACCELEROMETER_AND_GYROSCOPE", 0, "001", 255)
- <span style="color:orange">("ACCELEROMETER_AND_GYROSCOPE_DMP", 0, {Digital Low Pass (any 3 bit number other than "000", "111")}, {Sample rate divisor})</span>
    * Example - ("ACCELEROMETER_AND_GYROSCOPE_DMP", 0, "101", 0)
- <span style="color:orange">("GYROSCOPE", 0, {Digital Low Pass}, {Sample rate divisor})</span>
    * Example - ("GYROSCOPE", 0, "111", 11)
- <span style="color:orange">("GYROSCOPE_DMP", 0, {Digital Low Pass}, {Sample rate divisor})</span>
    * Example - ("GYROSCOPE_DMP", 0, "001", 60)

 
You should also choose how long the sensor will spend in that configuration (duration, in seconds) and how fast the MPU6000 sensor should retrieve samples (in seconds between samples).
- <span style="color:orange">**Configuration duration (s)**</span>: {n} 
- <span style="color:orange">**Sampling rate (s)**</span>: {n}
    - The sampling rate can be no smaller than the hardware sampling rate of the accelerometer, which is given by 
        * Hardware Sample Rate = 1/((Gyroscope Output Rate * 1000) / (1 + Sample Rate Divisor)) * 1000
        * Note that the extra 1000 on the end is for an overestimation. This may change as testing is done to confirm the hardware sampling rate


    
<span style="color:#18BF7D">For the following code, replace 'None' with appropriate values based on what you have learned above. Any lines without a 'None' should not be modified. Changing the values of the variables allows you to adjust a configuration consisting of a collection of these variables. At the end of the code, these configurations, in addition to their durations and the sampling rates, are put together in a list.</span>

In [ ]:
import micropip
await micropip.install("matplotlib")
import matplotlib.pyplot as plt
await micropip.install("numpy")
import numpy as np
from source.MPU6000 import MPU6000

# Example parameters

mode = "ACCELEROMETER_AND_GYROSCOPE"
low_power_wakeup = 0
digital_low_pass = "100"
sampling_rate_divsor = 75

example_configuration = (mode, low_power_wakeup, digital_low_pass, sampling_rate_divsor) # Parentheses are used to group variables together in a 'tuple', a data type in Python 
duration = 40
sampling_rate = 75


# Try creating 3 distinct configurations below by replacing None with appropriate values

# Configuration 1

mode = None
low_power_wakeup = None
digital_low_pass = None
sampling_rate_divsor = None

configuration_1 = (mode, low_power_wakeup, digital_low_pass, sampling_rate_divsor)
duration_1 = None
sampling_rate_1 = None

# Configuration 2

mode = None
low_power_wakeup = None
digital_low_pass = None
sampling_rate_divsor = None

configuration_2 = (mode, low_power_wakeup, digital_low_pass, sampling_rate_divsor)
duration_2 = None
sampling_rate_2 = None

# Configuration 3

mode = None
low_power_wakeup = None
digital_low_pass = None
sampling_rate_divsor = None

configuration_3 = (mode, low_power_wakeup, digital_low_pass, sampling_rate_divsor)
duration_3 = None
sampling_rate_3 = None


# Below, we group together the configuration tuple with a duration and sampling rate in another tuple
# These tuples are added to a list called modes_ACC
modes_ACC = [(configuration_1, duration_1, sampling_rate_1), (configuration_2, duration_2, sampling_rate_2), (configuration_3, duration_3, sampling_rate_3)]

## <span style="color:tomato"> Configurations for a Lunar Mission </span>

Now that you have explored different configurations, let's consider choosing the ideal mode for a lunar mission. The Accelerometer and Gyroscope mode and, Gyroscope Only mode is not appropriate for the mission because we only need to measure the acceleration of the LunaSat. In other words, using Accelerometer and Gyroscope and, Gyroscope Only wastes power and data usage. So we should only consider using the Accelerometer Only, Accelerometer Low Power or the Shutdown mode.

<span style="color:#18BF7D">With these limitations in mind, revisit the configurations you set above and revise them so that they are well-suited for a lunar mission. </span>

## <span style="color:tomato"> Calculating Power Usage </span>

We can calculate the power usage based on the configuration to get an idea of how it's affected by the various settings. Power is measured in milliWatts, where 1 milliWatt is equivalent to 0.001 Joules of energy per second. 


<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
power_1 = MPU6000.get_mode_power(MPU6000,*configuration_1, sampling_rate_1)
power_2 = MPU6000.get_mode_power(MPU6000,*configuration_2, sampling_rate_2)
power_3 = MPU6000.get_mode_power(MPU6000,*configuration_3, sampling_rate_3)
print("Config. 1: ", power_1, "milliWatts")
print("Config. 2: ", power_2, "milliWatts")
print("Config. 3: ", power_3, "milliWatts")

plt.bar(1, power_1, label='Config. 1')
plt.bar(2, power_2, label='Config. 2')
plt.bar(3, power_3, label='Config. 3')
plt.xticks([])
plt.ylabel('Power in mW')
plt.grid()
plt.legend()
plt.show()

## <span style="color:tomato"> Calculating Data Usage </span>

We can calculate the data usage for the configuration set above. It's important to note that we are concerned with the data usage based on the sampling rate.

The "time_step" is the distance between data values in the plot, and the total duration is how long the sensor is active for in the model.

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
time_step = 1
total_duration = duration_1 + duration_2 + duration_3 # total time to loop through configurations. Change if desired
mpu = MPU6000(time_step, total_duration, modes_ACC, loop_rate=1000)

In [ ]:
data_1 = mpu.get_bytes_per_second(*configuration_1,sampling_rate_1)
data_2 = mpu.get_bytes_per_second(*configuration_2,sampling_rate_2)
data_3 = mpu.get_bytes_per_second(*configuration_3,sampling_rate_3)

print("Config. 1: ", data_1, "bytes per second")
print("Config. 2: ", data_2, "bytes per second")
print("Config. 3: ", data_3, "bytes per second")

plt.bar(1, data_1, label='Config. 1')
plt.bar(2, data_2, label='Config. 2')
plt.bar(3, data_3, label='Config. 3')
plt.xticks([])
plt.ylabel('Bytes/s')
plt.grid()
plt.legend()
plt.show()

## <span style="color:tomato"> Putting It All Together </span>

Finally, we'll take a look at the power and data usage of all of our configurations together. 

<span style="color:#18BF7D">Do not modify the following code. Simply run the cell to see the output.</span>

In [ ]:
acc_power, acc_data, acc_time = mpu.run_sim()

We'll store our generated data for use in the combined power and data model.

In [ ]:
%store acc_power
%store acc_data
%store acc_time

[Click here to continue to the next sensor, the thermopile.](1.3%20Thermopile.ipynb)

If you would like to look at more detailed information for the MPU6000 sensor, consider looking at the [MPU6000 Datasheet](https://invensense.tdk.com/wp-content/uploads/2015/02/MPU-6000-Datasheet1.pdf)

[MPU6000 Register Map](https://invensense.tdk.com/wp-content/uploads/2015/02/MPU-6000-Register-Map1.pdf)